In [1]:
import lmdb
import os
import copy
import torch
import random
import skimage
import numpy as np
import torch.nn as nn
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

print("complete")

complete


In [2]:
train_vectors = np.load("train_vectors_bird.npy")
test_vectors = np.load("test_vectors_bird.npy")
local_train = '/home/lvlab/Pytorch/train_vectors_bird.npy'
local_test = '/home/lvlab/Pytorch/test_vectors_bird.npy'

In [3]:
def pro_select(vectors, probs, num):
    points = np.zeros((num, 2))
    core_point = np.zeros((1, 2))
    for i in range(num):
        x = random.uniform(0, 1)
        cumprob = 0.0
        k = -1
        while x > cumprob:
            k += 1
            cumprob += probs[k]
        points[i, 0], points[i, 1] = vectors[k, 0], vectors[k, 1]
        
        adj_list = []
        core_point[0, 0], core_point[0, 1] = vectors[k, 0], vectors[k, 1]
        dis = np.sum(pow(vectors - core_point, 2), axis=1)
        for j in range(len(dis)):
            if dis[j] <= 8:
                adj_list.append(j)
        for j in range(len(adj_list)):
            probs[adj_list[j]] /= 100.0
            
        probs[k] = 0
        probs = probs/np.sum(probs)
        
    return points

class Bird_Test_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, local_file, transform=None, 
                 transform2=None, transform3=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.transform2 = transform2
        self.transform3 = transform3
        self.train_vectors = train_vectors
        self.part_num = num
        self.local_points = np.load(local_file)

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        sample = {}
        sample['image'] = copy.deepcopy(image)
        if self.transform2:
            sample['image'] = self.transform2(sample['image'])

        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        
        vectors = self.local_points[idx]
        x1, x2, y1, y2 = np.min(vectors[:, 0]), np.max(vectors[:, 0]), \
                        np.min(vectors[:, 1]), np.max(vectors[:, 1])
        edge = 16
        m1, m2, n1, n2 = int(np.maximum(x1 * 16 + 8 - edge, 0)), int(np.minimum(x2 * 16 + 8 + edge, 448)), \
                        int(np.maximum(y1 * 16 + 8 - edge, 0)), int(np.minimum(y2 * 16 + 8 + edge, 448))
        obj = image[m1:m2, n1:n2]
        sample['object'] = copy.deepcopy(obj)
        if self.transform3:
            sample['object'] = self.transform3(sample['object'])
            

        vectors = self.train_vectors[idx]
        
        points = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)
        
        points1 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)
        
        points2 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
            x1, y1 = points1[i][0] * 16 + 8, points1[i][1] * 16 + 8
            l1 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x1 - l1)), int(np.minimum(448, x1 + l1)), \
                            int(np.maximum(0, y1 - l1)), int(np.minimum(448, y1 + l1))
            sample['s_image%d' % i] = image[m1:m2, n1:n2]
            x2, y2 = points2[i][0] * 16 + 8, points2[i][1] * 16 + 8
            l2 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x2 - l2)), int(np.minimum(448, x2 + l2)), \
                            int(np.maximum(0, y2 - l2)), int(np.minimum(448, y2 + l2))
            sample['t_image%d' % i] = image[m1:m2, n1:n2]
            
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])
                sample['s_image%d' % i] = self.transform(sample['s_image%d' % i])
                sample['t_image%d' % i] = self.transform(sample['t_image%d' % i])

        return sample
    
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, local_file, transform=None, transform2=None, 
                 transform3=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num
        self.local_points = np.load(local_file)
        self.transform2 = transform2
        self.transform3 = transform3

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
        
        sample = {}
        sample['image'] = copy.deepcopy(image)
        if self.transform:
            sample['image'] = self.transform2(sample['image'])
        
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        
        vectors = self.local_points[idx]
        x1, x2, y1, y2 = np.min(vectors[:, 0]), np.max(vectors[:, 0]), \
                        np.min(vectors[:, 1]), np.max(vectors[:, 1])
        edge = 16
        m1, m2, n1, n2 = int(np.maximum(x1 * 16 + 8 - edge, 0)), int(np.minimum(x2 * 16 + 8 + edge, 448)), \
                        int(np.maximum(y1 * 16 + 8 - edge, 0)), int(np.minimum(y2 * 16 + 8 + edge, 448))
        obj = image[m1:m2, n1:n2]
        sample['object'] = copy.deepcopy(obj)
        if self.transform3:
            sample['object'] = self.transform3(sample['object'])
        
        
        vectors = self.train_vectors[idx]
        points = pro_select(vectors[:, :2].astype(int), vectors[:, 3], self.part_num)
        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [4]:
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

data_transform3 = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize([224, 224]),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
    ]),
}

root_dir = '/home/lvlab/Pytorch/images'
train_file = '/home/lvlab/Pytorch/train.txt'
test_file = '/home/lvlab/Pytorch/test.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, local_train, 
                                          data_transform, data_transforms['train'], data_transform3),
                  'test': Bird_Test_Process(test_file, root_dir, test_vectors, 4, local_test, 
                                            data_transform, data_transforms['test'], data_transform3)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=20,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
# print(img_data['test'][0])

In [6]:
class union_model(nn.Module):
    def __init__(self, model_part, model_obj, model_img):
        super(union_model, self).__init__()
        self.model_part = model_part
        self.model_obj = model_obj
        self.model_img = model_img

    def forward(self, x, flag, x_s=None, x_t=None):
        y0 = self.model_part(x[0]) + self.model_part(x[1]) + \
            self.model_part(x[2]) + self.model_part(x[3])
        y1 = self.model_obj(x[4])
        y2 = self.model_img(x[5])
        if flag:
            y0 = (self.model_part(x_s[0]) + self.model_part(x_s[1]) + \
                    self.model_part(x_s[2]) + self.model_part(x_s[3]) + \
                    self.model_part(x_t[0]) + self.model_part(x_t[1]) + \
                    self.model_part(x_t[2]) + self.model_part(x_t[3]) + \
                    y0)/3.0
            
        output = y0 + y1 + y2

        return output

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
                
        # Each epoch has a training and validation phase
        for phase in ['test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(loader_img[phase]):
                img0 = sample_batched['image0']
                img1 = sample_batched['image1']
                img2 = sample_batched['image2']
                img3 = sample_batched['image3']
                obj = sample_batched['object']
                original = sample_batched['image']
                labels = sample_batched['label']
                if phase == 'test':
                    s_img0 = sample_batched['s_image0']
                    s_img1 = sample_batched['s_image1']
                    s_img2 = sample_batched['s_image2']
                    s_img3 = sample_batched['s_image3']
                    t_img0 = sample_batched['t_image0']
                    t_img1 = sample_batched['t_image1']
                    t_img2 = sample_batched['t_image2']
                    t_img3 = sample_batched['t_image3']
                    s_img0 = s_img0.to(device)
                    s_img1 = s_img1.to(device)
                    s_img2 = s_img2.to(device)
                    s_img3 = s_img3.to(device)
                    t_img0 = t_img0.to(device)
                    t_img1 = t_img1.to(device)
                    t_img2 = t_img2.to(device)
                    t_img3 = t_img3.to(device)
                
                obj = obj.to(device)
                original = original.to(device)
                img0 = img0.to(device)
                img1 = img1.to(device)
                img2 = img2.to(device)
                img3 = img3.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        imgs = [img0, img1, img2, img3, obj, original]
                        output = model(imgs, False)
                    else:
                        imgs = [img0, img1, img2, img3, obj, original]
                        s_imgs = [s_img0, s_img1, s_img2, s_img3]
                        t_imgs = [t_img0, t_img1, t_img2, t_img3]
                        output = model(imgs, True, s_imgs, t_imgs)

                    _, preds = torch.max(output, 1)
                    loss = criterion(output, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * img3.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:1")

model_part = torch.load('/home/lvlab/Pytorch/random_part_bird')
model_obj = torch.load('/home/lvlab/Pytorch/object_bird', map_location={'cuda:2':'cuda:1'})
model_original = torch.load('/home/lvlab/Pytorch/image_bird')

random_model = union_model(model_part, model_obj, model_original)

random_model = random_model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(random_model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=5)

model = train_model(random_model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
test Loss: 2.1471 Acc: 0.8454
test 1 epoch time: 4.99min

Training complete in 5m 3s
Best val Acc: 0.845357
